<a href="https://colab.research.google.com/github/Themis-Sot/datam/blob/main/DataMining_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Αρχικοποίηση


In [39]:
import matplotlib
import sys
import pandas
import random
from google.colab import drive


# drive.mount('/content/gdrive')
data=pandas.read_csv('/content/gdrive/MyDrive/DataMining/Project1/data_sample/netflix_titles.csv')
data.columns=['show_id','type','title','director','cast','country','date_added','release year','rating','duration','listed_in','description']
data=data.drop(labels=0,axis=0)
data.head(5)

,show_id,type,title,director,cast,country,date_added,release year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...


**Ερώτημα 1:**
*Ποιό είδος υπερτερεί, οι ταινίες ή οι σειρές ;*


In [40]:
num_movies=data.loc[data['type']=='Movie'].count().max()
num_shows=data.loc[data['type']=='TV Show'].count().max()
if num_movies>num_shows:
    print('Υπερτερούν οι ταινίες με αριθμό:', num_movies)
else:
    print('Υπερτερούν οι σειρές με αριθμό', num_show)

Υπερτερούν οι ταινίες με αριθμό: 5377


**Ερώτημα 2:**
*Τα τελευταία χρόνια το netflix επενδύει περισσότερο σε ταινίες ή σε σειρές;*

In [41]:
print("Ερώτημα 2:")

Ερώτημα 2:


**Ερώτημα 3:**
*Ποιά χώρα έχει το περισσότερο περιεχόμενο;*

In [42]:
country=data['country'].value_counts()[:1].index.tolist()
output=''.join(country)
print("Η χώρα με το περισσότερο περιεχόμενο είναι: ",output)

Η χώρα με το περισσότερο περιεχόμενο είναι:  United States


**Ερώτημα 4:**
*Τι είδους περιεχόμενο έχει κάθε χώρα;*

In [45]:
# countries=set()
# for country in data['country']:
#     countries.add(country)
# print(countries)

countries=set()
tok=data['country'].astype(str).str.split(",")
print(tok)

# countries=data['country'].astype(str).str.split(",").tolist()
# print(countries)

# countries=data['country'].astype(str).tolist()
# for c in countries:
#     print(c)

1                                                [Mexico]
2                                             [Singapore]
3                                         [United States]
4                                         [United States]
5                                                [Turkey]
                              ...                        
7782    [Sweden,  Czech Republic,  United Kingdom,  De...
7783                                              [India]
7784                                                [nan]
7785                                          [Australia]
7786            [United Kingdom,  Canada,  United States]
Name: country, Length: 7786, dtype: object
